# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk 

nltk.download(['punkt', 'wordnet','stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
import re
import numpy as np
import pandas as pd
import sqlite3
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
X = df['message']
Y = df.iloc[:,4:]

In [5]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    #1- apply normalize step on text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    #2- apply tokenize by Split text into words using NLTK
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    #apply sterm and lemmatizer
    lemmatizer = WordNetLemmatizer()
    new_tokens = []
    for i in tokens:
        tok = lemmatizer.lemmatize(i).strip()
        new_tokens.append(tok)
        
    return new_tokens
#test tokenize function
print(df.loc[5, "message"]+'\n', tokenize(df.loc[5, "message"]))


Information about the National Palace-
 ['information', 'national', 'palace']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y) 
pipeline.fit(X_train, y_train) 
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def test_model(y_test, y_pred):
    for i in range(y_test.shape[1]):
        print("Classification Report Of" + y_test.columns[i]+'\n'
              ,'\n'+ classification_report(y_test.values[:,i],y_pred[:,i])
              , 'Accuracy:', accuracy_score(y_test.values[:, i], y_pred[:, i]))
        
test_model(y_test, y_pred)

Classification Report Ofrelated
 
             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1493
          1       0.85      0.92      0.88      5018
          2       0.33      0.44      0.38        43

avg / total       0.79      0.81      0.80      6554
 Accuracy: 0.806988098871
Classification Report Ofrequest
 
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5429
          1       0.79      0.44      0.56      1125

avg / total       0.88      0.88      0.87      6554
 Accuracy: 0.883582545011
Classification Report Ofoffer
 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6513
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      6554
 Accuracy: 0.993744278303
Classification Report Ofaid_related
 
             precision    recall  f1-score   support

          0       0.74      0.85   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__min_samples_split': [2, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=4)

In [11]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1) ...
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 8.7min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 8.7min
[CV] clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 1), total= 8.8min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=2, vect__ngram_range=(1, 2), total=16.2min
[CV] clf__estimator__min_samples_split=4, vect__ngram_range=(1, 1) ...
[CV]  clf__estimator__min_samples_split=

[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 46.7min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 46.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred = cv.predict(X_test)
test_model(y_test, y_pred)
print("\nParameters:", cv.best_params_)

Classification Report Ofrelated
 
             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1493
          1       0.84      0.92      0.88      5018
          2       0.69      0.21      0.32        43

avg / total       0.79      0.81      0.79      6554
 Accuracy: 0.805462313091
Classification Report Ofrequest
 
             precision    recall  f1-score   support

          0       0.89      0.97      0.93      5429
          1       0.77      0.44      0.56      1125

avg / total       0.87      0.88      0.87      6554
 Accuracy: 0.881904180653
Classification Report Ofoffer
 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6513
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      6554
 Accuracy: 0.993744278303
Classification Report Ofaid_related
 
             precision    recall  f1-score   support

          0       0.75      0.85   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
pipeline = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa17f2c5f28>, use_idf=True,
           vocabulary=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), no

In [14]:
parameters = {
    'vect__smooth_idf': [True,False],
}
cv = GridSearchCV(pipeline, param_grid=parameters,n_jobs=-1)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__smooth_idf': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
y_pred = cv.predict(X_test)   
test_model(y_test, y_pred)

Classification Report Ofrelated
 
             precision    recall  f1-score   support

          0       0.66      0.10      0.18      1493
          1       0.78      0.98      0.87      5018
          2       0.29      0.12      0.17        43

avg / total       0.75      0.78      0.71      6554
 Accuracy: 0.776319804699
Classification Report Ofrequest
 
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5429
          1       0.76      0.53      0.62      1125

avg / total       0.88      0.89      0.88      6554
 Accuracy: 0.889533109551
Classification Report Ofoffer
 
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6513
          1       0.00      0.00      0.00        41

avg / total       0.99      0.99      0.99      6554
 Accuracy: 0.992218492524
Classification Report Ofaid_related
 
             precision    recall  f1-score   support

          0       0.76      0.86   

### 9. Export your model as a pickle file

In [18]:
with open('ml_pipeline_model.pkl', 'wb') as file:
    pickle.dump(cv, file)

NameError: name 'ml_pipeline' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.